In [1]:
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.time import Time
import numpy as np
from astropy.utils.data import download_file

def ztf_cone_search(pos):
    """
    Query the ZTF database for observations near a given position.
    
    Parameters:
    pos (SkyCoord): The position to query (RA, Dec).

    Returns:
    np.ndarray: Structured array of observation results.
    """
    # Build the URL for the ZTF database query
    url = f"https://irsa.ipac.caltech.edu/ibe/search/ztf/products/sci?POS={pos.ra.deg},{pos.dec.deg}"
    fname = download_file(url, timeout=300, cache=True)

    lines = []
    with open(fname, 'r') as fp:
        for line in fp:
            if line[0] == "\\":
                # Skip comment lines
                continue
            lines.append(line)
    
    # Parse column names and their positions
    colnames = [col.strip() for col in lines[0].strip()[1:-1].split('|')]
    colmarkers = [0]
    ind = 0
    while ind != -1:
        ind = lines[0].find('|', ind + 1)
        if ind != -1:
            colmarkers.append(ind)

    # Define data types for each field
    ctypes = {'int': int, 'long': int, 'float': float, 'double': float, 'char': "U24"}
    ctypes2 = ctypes.copy()
    ctypes2["char"] = str
    coltypes = [ctypes[ctype.strip()] for ctype in lines[1].strip()[1:-1].split('|')]
    coltypes2 = [ctypes2[ctype.strip()] for ctype in lines[1].strip()[1:-1].split('|')]
    dtype = [(colname, coltype) for colname, coltype in zip(colnames, coltypes)]

    # Parse data rows
    rows = []
    for index, line in enumerate(lines):
        if index < 4:
            # Skip header rows
            continue

        row = [coltypes2[icol](line[colmarkers[icol] + 1:colmarkers[icol + 1]].strip()) 
               for icol in range(len(colnames))]
        rows.append(np.array(tuple(row), dtype=dtype))
    
    return np.array(rows)


In [ ]:
THIS CODE SAVES MAG AND FIELD IN CSV FILE 

In [5]:
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u

file_path = "/home/jovyan/research/Current/diego_data.csv"  # Update the path
data = pd.read_csv(file_path)

# Extract the first 100 supernovae to analyze all observations
selected_sn = data.iloc[:]

# Create a list to store observation information
obs_list = []

for idx, sn in selected_sn.iterrows():
    ztfid = sn['ZTFID']
    print(f"Processing observations for {ztfid}...")
    
    # Convert RA and Dec from sexagesimal to decimal degrees
    pos = SkyCoord(ra=sn['RA'], dec=sn['Dec'], unit=(u.hourangle, u.deg))
    
    # Query ZTF for all observations
    observations = ztf_cone_search(pos)
    
    # Check if any observations exist using length
    if len(observations) > 0:
        for entry in observations:
            obs_jd = entry['obsjd']  # Julian Date of observation
            field = entry['field']
            maglimit = entry['maglimit']  # Corrected from limmag
            
            # Append observation details to the list
            obs_list.append({
                "ZTFID": ztfid,
                "obsjd": obs_jd,
                "field": field,
                "maglimit": maglimit
            })
        print(f"Finished processing observations for {ztfid}.")
    else:
        print(f"No observations found for {ztfid}.")

# Convert the list of observations into a DataFrame and save as CSV
output_df = pd.DataFrame(obs_list)
output_df.to_csv("/home/jovyan/research/Current/SNfield_mag.csv", index=False)



Processing observations for ZTF17aabtvsy...
Finished processing observations for ZTF17aabtvsy.
Processing observations for ZTF17aabvong...
Finished processing observations for ZTF17aabvong.
Processing observations for ZTF17aacldgo...
Finished processing observations for ZTF17aacldgo.
Processing observations for ZTF17aadlxmv...
Finished processing observations for ZTF17aadlxmv.
Processing observations for ZTF18aaaonon...
Finished processing observations for ZTF18aaaonon.
Processing observations for ZTF18aaaqexr...
Finished processing observations for ZTF18aaaqexr.
Processing observations for ZTF18aabkrky...
Finished processing observations for ZTF18aabkrky.
Processing observations for ZTF18aadlaxo...
Finished processing observations for ZTF18aadlaxo.
Processing observations for ZTF18aadxnul...
Finished processing observations for ZTF18aadxnul.
Processing observations for ZTF18aadzfso...
Finished processing observations for ZTF18aadzfso.
Processing observations for ZTF18aaeqjmc...
Finish

In [ ]:
THIS CREATES A DIRECTORY AND PLACES THE UNIQUE FIELD NUMBER 

In [6]:
import os
import numpy as np
import pandas as pd

# Create main directory "Field_Numbers"
main_dir = "Field_Numbers"
os.makedirs(main_dir, exist_ok=True)

# Read the CSV file containing observation data
csv_file_path = "/home/jovyan/research/Current/SNfield_mag.csv"
df = pd.read_csv(csv_file_path)

# Assuming the CSV file has columns: 'obsjd', 'maglimit', and 'field'
# Get unique field numbers using numpy unique
unique_fields = np.unique(df['field'].values)
print("Unique fields found:", unique_fields)

# Process each unique field
for field in unique_fields:
    # Create a directory for the field under "Field_Numbers"
    field_dir = os.path.join(main_dir, str(field))
    os.makedirs(field_dir, exist_ok=True)
    
    # Define the output CSV file path within the field's directory
    out_csv = os.path.join(field_dir, "observation.csv")
    
    # Filter the DataFrame for the current field and select the relevant columns
    field_df = df[df['field'] == field][['obsjd', 'maglimit']]
    
    # Sort the DataFrame by Julian date in chronological order
    field_df = field_df.sort_values(by='obsjd')
    
    # If the CSV file already exists, read it, append the new data, sort, and write back
    if os.path.exists(out_csv):
        existing_df = pd.read_csv(out_csv)
        combined_df = pd.concat([existing_df, field_df], ignore_index=True)
        combined_df = combined_df.sort_values(by='obsjd')
    else:
        combined_df = field_df
    
    # Save the combined DataFrame to the CSV file
    combined_df.to_csv(out_csv, index=False)
    print(f"Saved {len(combined_df)} observations for field {field} in {out_csv}")


Unique fields found: [ 245  246  247 ... 1892 1893 1896]
Saved 3848 observations for field 245 in Field_Numbers/245/observation.csv
Saved 5315 observations for field 246 in Field_Numbers/246/observation.csv
Saved 3777 observations for field 247 in Field_Numbers/247/observation.csv
Saved 2323 observations for field 248 in Field_Numbers/248/observation.csv
Saved 894 observations for field 249 in Field_Numbers/249/observation.csv
Saved 1028 observations for field 250 in Field_Numbers/250/observation.csv
Saved 1066 observations for field 251 in Field_Numbers/251/observation.csv
Saved 1179 observations for field 252 in Field_Numbers/252/observation.csv
Saved 1576 observations for field 253 in Field_Numbers/253/observation.csv
Saved 1415 observations for field 254 in Field_Numbers/254/observation.csv
Saved 1442 observations for field 255 in Field_Numbers/255/observation.csv
Saved 3271 observations for field 256 in Field_Numbers/256/observation.csv
Saved 5057 observations for field 257 in Fie

In [7]:
print("Total unique fields:", len(unique_fields))


Total unique fields: 1076
